In [224]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai

In [225]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [226]:
genai.configure(api_key=gemini_api_key)

In [227]:
context = {}

In [228]:
employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [229]:
context.keys()

dict_keys(['Chen', 'Harper', 'Thomson', 'Lancaster', 'Carter', 'Tran', 'Blake', 'Bishop', 'Thompson', 'Spencer', 'Greene', 'Trenton'])

In [230]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [231]:
context.keys()

dict_keys(['Chen', 'Harper', 'Thomson', 'Lancaster', 'Carter', 'Tran', 'Blake', 'Bishop', 'Thompson', 'Spencer', 'Greene', 'Trenton', 'Carllm', 'Homellm', 'Markellm', 'Rellm'])

In [232]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context

In [233]:
get_relevant_context("Who is lancaster?")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

In [234]:
get_relevant_context("What is Alex Chen date of birth?")

['# HR Record\n\n# Alex Chen\n\n## Summary\n- **Date of Birth:** March 15, 1990  \n- **Job Title:** Backend Software Engineer  \n- **Location:** San Francisco, California  \n\n## Insurellm Career Progression\n- **April 2020:** Joined Insurellm as a Junior Backend Developer. Focused on building APIs to enhance customer data security.\n- **October 2021:** Promoted to Backend Software Engineer. Took on leadership for a key project developing a microservices architecture to support the company\'s growing platform.\n- **March 2023:** Awarded the title of Senior Backend Software Engineer due to exemplary performance in scaling backend services, reducing downtime by 30% over six months.\n\n## Annual Performance History\n- **2020:**  \n  - Completed onboarding successfully.  \n  - Met expectations in delivering project milestones.  \n  - Received positive feedback from the team leads.\n\n- **2021:**  \n  - Achieved a 95% success rate in project delivery timelines.  \n  - Awarded "Rising Star" 

In [235]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [236]:
print(add_context("What is Alex Chen date of birth?"))

What is Alex Chen date of birth?

The following additional context might be relevant in answering this question:

# HR Record

# Alex Chen

## Summary
- **Date of Birth:** March 15, 1990  
- **Job Title:** Backend Software Engineer  
- **Location:** San Francisco, California  

## Insurellm Career Progression
- **April 2020:** Joined Insurellm as a Junior Backend Developer. Focused on building APIs to enhance customer data security.
- **October 2021:** Promoted to Backend Software Engineer. Took on leadership for a key project developing a microservices architecture to support the company's growing platform.
- **March 2023:** Awarded the title of Senior Backend Software Engineer due to exemplary performance in scaling backend services, reducing downtime by 30% over six months.

## Annual Performance History
- **2020:**  
  - Completed onboarding successfully.  
  - Met expectations in delivering project milestones.  
  - Received positive feedback from the team leads.

- **2021:**  
  

In [248]:
model = genai.GenerativeModel('gemini-2.0-flash')

def chat(message, history):
    system_prompt = (
        "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. "
        "Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up "
        "if you haven't been provided with relevant context."
    )

    chat_history_str = ""
    for user, bot in history:
        chat_history_str += f"User: {user}\nAssistant: {bot}\n"

    user_message = add_context(message)

    full_input = f"{system_prompt}\n\n{chat_history_str}User: {user_message}\nAssistant:"

    response_stream = model.generate_content(full_input, stream=True, generation_config={"temperature": 0})

    partial_answer = ""
    for chunk in response_stream:
        if chunk.text:
            partial_answer += chunk.text
            yield partial_answer  

    history.append((message, partial_answer))
    
    print(f"History: {history}")


In [ ]:
gr.ChatInterface(chat, title="Insurellm Gemini QA Assistant").launch()

c:\Users\chandula\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


History: [('Tell me about Alex Chen', 'Alex Chen is a Backend Software Engineer at Insurellm in San Francisco, California. He joined in April 2020 as a Junior Backend Developer and was promoted to Backend Software Engineer in October 2021 and then to Senior Backend Software Engineer in March 2023. He has consistently received positive performance reviews and has been awarded for his contributions, including the "Rising Star" award in 2021. He is involved in Diversity & Inclusion initiatives and is pursuing further professional development in AI and machine learning.\n')]
History: [['Tell me about Alex Chen', 'Alex Chen is a Backend Software Engineer at Insurellm in San Francisco, California. He joined in April 2020 as a Junior Backend Developer and was promoted to Backend Software Engineer in October 2021 and then to Senior Backend Software Engineer in March 2023. He has consistently received positive performance reviews and has been awarded for his contributions, including the "Rising